In [1]:
import tensorflow as tf
import cv2 as cv
import numpy
from matplotlib import pyplot as plt
from matplotlib import cm
label =['person',
'bicycle',
'car',
'motorcycle',
'airplane',
'bus',
'train',
'truck',
'boat',
'traffic light',
'fire hydrant',
'street sign',
'stop sign',
'parking meter',
'bench',
'bird',
'cat',
'dog',
'horse',
'sheep',
'cow',
'elephant',
'bear',
'zebra',
'giraffe',
'hat',
'backpack',
'umbrella',
'shoe',
'eye glasses',
'handbag',
'tie',
'suitcase',
'frisbee',
'skis',
'snowboard',
'sports ball',
'kite',
'baseball bat',
'baseball glove',
'skateboard',
'surfboard',
'tennis racket',
'bottle',
'plate',
'wine glass',
'cup',
'fork',
'knife',
'spoon',
'bowl',
'banana',
'apple',
'sandwich',
'orange',
'broccoli',
'carrot',
'hot dog',
'pizza',
'donut',
'cake',
'chair',
'couch',
'potted plant',
'bed',
'mirror',
'dining table',
'window',
'desk',
'toilet',
'door',
'tv',
'laptop',
'mouse',
'remote',
'keyboard',
'cell phone',
'microwave',
'oven',
'toaster',
'sink',
'refrigerator',
'blender',
'book',
'clock',
'vase',
'scissors',
'teddy bear',
'hair drier',
'toothbrush',
'hair brush',]



In [2]:
points = []
def get_points(x1, y1, x2, y2):

    for x in range(x1, x2,50):
        for y in range(y1, y2,50):
            points.append((x, y))
                  
    return points


In [19]:
def final_depth(points):
    final_depth = []
    left  = cv.imread("D:\\Backpack-perfect\\Backpack-perfect\\im0.png", cv.IMREAD_GRAYSCALE)
    right = cv.imread("D:\\Backpack-perfect\\Backpack-perfect\\im1.png", cv.IMREAD_GRAYSCALE)

    fx = 7190.247        # lense focal length
    baseline = 174.945  # distance in mm between the two cameras
    disparities = 256 # num of disparities to consider
    block = 31        # block size to match
    units = 0.512     # depth units, adjusted for the output to fit in one byte
    doffs=342.523

    sbm = cv.StereoBM_create(numDisparities=disparities,
                              blockSize=block)

    # disparities
    disparity = sbm.compute(left, right)
    valid_pixels = disparity > 0


    depth = numpy.zeros(shape=left.shape).astype("uint8")
    depth[valid_pixels] = (fx * baseline) / ((disparity[valid_pixels])*units)
    
    for (x,y) in points:

        if((x<valid_pixels.shape[0]) and (y<valid_pixels.shape[1]) and (valid_pixels[x][y])):
            
            val = (fx * baseline) / (disparity[x][y]+doffs)
            print(val)
            final_depth.append(val)

    avg_depth = sum(final_depth)/len(final_depth)

    depth = cv.equalizeHist(depth)
    colorized_depth = numpy.zeros((left.shape[0], left.shape[1], 3), dtype="uint8")
    temp = cv.applyColorMap(depth, cv.COLORMAP_JET)
    colorized_depth[valid_pixels] = temp[valid_pixels]
    cv.imwrite("p.jpg",colorized_depth)
    (unique, counts) = numpy.unique(final_depth, return_counts=True)
    frequencies = numpy.asarray((unique, counts)).T
    return frequencies, avg_depth

In [21]:
with tf.gfile.FastGFile('frozen_inference_graph.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Session() as sess:

    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')

    img = cv.imread('D:\\Backpack-perfect\\Backpack-perfect\\im0.png')
    rows = img.shape[0]
    cols = img.shape[1]
    inp = cv.resize(img, (2016, 2940))
    inp = inp[:, :, [2, 1, 0]]  

    
    out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                    sess.graph.get_tensor_by_name('detection_scores:0'),
                    sess.graph.get_tensor_by_name('detection_boxes:0'),
                    sess.graph.get_tensor_by_name('detection_classes:0')],
                   feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})
    
    num_detections = int(out[0][0])
    
    for i in range(num_detections):
        classId = int(out[3][0][i])
        score = float(out[1][0][i])
        bbox = [float(v) for v in out[2][0][i]]
    
        if score > 0.3:
            x = bbox[1] * cols
            y = bbox[0] * rows
            right = bbox[3] * cols
            bottom = bbox[2] * rows
            cv.rectangle(img, (int(x), int(y)), (int(right), int(bottom)), (0, 0, 0), thickness=2)
            ##print((int(x), int(y)))  top left coordinate
            ##print((int(right), int(bottom))) bottom right coordinate
            
            width = int(right)-int(x)
            height = int(bottom)-int(y)
            print(label[classId-1])
            points = get_points(int(x), int(y), int(right), int(bottom))
            depth, avg_depth = final_depth(points)
            print('Average dist in mm',avg_depth)

            
cv.imshow('Image', img)
cv.waitKey()

backpack
418.3895354916626
418.6680419537477
416.4503171851365
413.30318890805165
411.8147944589057
812.8459230751336
405.18229738191667
404.0110708721278
401.0484735533583
399.3931021983329
395.2517425372888
391.07376549615844
389.8617060578834
385.0876792892626
384.26421974582126
381.12073796031723
419.36593298834515
418.3895354916626
415.8995522318726
414.2559636185861
411.54532827497127
407.67732452974747
405.96689500610455
404.27075789412453
401.43243289262597
399.3931021983329
393.1516546107029
393.1516546107029
391.31708232138953
387.5793705405878
387.1022797545978
384.4991343221491
381.93076575296425
380.199189008207
378.14191016115024
376.33181923205916
375.8820009350003
371.1135051790473
370.5668694585366
410.6049673578426
420.34689839142425
417.5562348951361
415.8995522318726
414.1195840870999
409.5355175315308
408.07408782189134
406.49165038198134
403.1047876958446
401.9455237795025
399.7738969062041
396.12302018124257
394.8795100255123
391.9267010752065
390.58803846921757


420.34689839142425
417.5562348951361
415.8995522318726
414.1195840870999
409.5355175315308
408.07408782189134
406.49165038198134
403.1047876958446
401.9455237795025
399.7738969062041
396.12302018124257
394.8795100255123
391.9267010752065
390.58803846921757
386.1516131781725
382.86073827971984
379.74008374130534
377.68775697240346
376.4444420747665
374.7621456534038
370.8946574783659
419.7857855304298
418.3895354916626
416.4503171851365
413.4390311642673
411.54532827497127
408.6043084345967
405.8359177896083
404.14087266664376
401.43243289262597
399.2663317852306
396.62261992582114
394.6317442776099
391.5607021070355
389.8617060578834
385.44167190333883
383.5611951539904
381.81483675026703
379.2820859119626
378.3693965765916
377.34785733141786
374.09342967022087
371.99148472892244
368.82840591164467
420.768718425983
417.8336326993682
416.1747524882358
414.2559636185861
411.4107273812822
407.8094932068913
405.7050250602882
404.14087266664376
401.560582768267
399.1396418223824
396.9981475

385.3236020744838
383.4442744083794
381.00530010392174
378.2556191657673
375.65749478650736
373.87105435599636
371.2230259068627
369.91302850032184
369.26149079721466
419.7857855304298
420.34689839142425
419.08649755973886
414.93921089003777
413.4390311642673
411.2762145045174
407.9417476097957
407.28133202019217
404.66091497955784
400.41017093142403
399.1396418223824
396.9981475327779
394.8795100255123
393.1516546107029
389.7409132065054
386.98319052503246
384.4991343221491
381.93076575296425
379.85475607900054
378.5971568639254
375.8820009350003
373.6489432613412
372.65270046004724
370.45773549906744
367.85766945126556
412.8961972107219
415.35024215269294
415.8995522318726
412.0846137490201
409.5355175315308
408.86993577327394
406.09795679160413
402.58873479727947
400.9206502757111
399.3931021983329
397.2488945809015
394.6317442776099
393.39756474464764
389.0177250679831
388.65713650575015
386.5075654450744
383.7952506862652
382.9773033755587
381.23624578916406
377.91469712391955
376

393.5390484570845
height 226082.87969268358
width 159464.86389711703


-1